In [1]:
%load_ext autoreload
%autoreload 2

from experiments.all import train_and_evaluate
from tsm.configuration import TSMConfigParser
import docker
import logging

logging.basicConfig(level=logging.INFO)

# load configuration
config = TSMConfigParser()
config.read("/home/pablo/tesis/tsm-pipeline/code/configs/pablo-bestia.cfg")
config.check()
docker_client = docker.from_env()

loading the dataset and creating a random state to make everything repeatable

In [2]:
from scripts.docker import read_dbs_dataset
from numpy.random import RandomState

xss_dbs = read_dbs_dataset("/home/pablo/tesis/tsm-pipeline/experiments/datasets/xss_1.txt")
# Using a fixed random state to have reproducible results
rand = RandomState(seed=5)

print(len(xss_dbs))

90


smoke testing run to check everything is working

In [ ]:
import tempfile

test_and_train = [
    xss_dbs[0],
]

res_dir = tempfile.mkdtemp()
print("using %s for smoke run" % (res_dir))
# !rm -rf $res_dir/import pandas as pd

concated_results = pd.concat(collected_results)
pd.concat([concated_results, concated_results.apply(['mean'])])*
import pandas as pd

concated_results = pd.concat(collected_results)
pd.concat([concated_results, concated_results.apply(['mean'])])
train_and_evaluate(
    config=config,
    results_dir=res_dir,
    query_type="xss",
    train=test_and_train,
    test=test_and_train,
    docker_client=docker_client,
    skip=["train", "evalute_worse", "evaluate_boosted", "evaluate_v0"]
)

as a second sanity, doing a run with 10/10 over the first 20 sample

In [ ]:
from random import sample
from sklearn.model_selection import train_test_split

sample_20 = sample(xss_dbs, 20)
train, test = train_test_split(sample_20, train_size=.5, shuffle=True, random_state=rand)
print("train %d, test %d" %(len(train), len(test)))

In [ ]:
res_dir = tempfile.mkdtemp()
print("using %s for 20 run" % (res_dir))

train_and_evaluate(
    config=config,
    results_dir=res_dir,
    query_type="xss",
    train=train,
    test=test,
    docker_client=docker_client,
)

igual que en path, la idea es hacer un corrida haciendo un k-fold pequeño para corrorar que el problema es el dataset

In [ ]:
from random import sample
from sklearn.model_selection import KFold
import numpy as np

sample_30 = sample(xss_dbs, 30)
print("using sample of %d" %(len(sample_30)))
# fixing random state to make this repeatable
kf = KFold(n_splits=3)
collected_results = []
base_results_dir = "/home/pablo/dev-results/xssrun2-30-3fold-"
for i, (train_index, test_index) in enumerate(kf.split(sample_30)):
    print("Fold %d" % (i))
    res_dir = "%s%d" % (base_results_dir, i)
    train_split = np.take(sample_30, train_index)
    test_split = np.take(sample_30, test_index)

    scores = train_and_evaluate(
        config=config,
        results_dir=res_dir,
        query_type="xss",
        train=train_split.tolist(),
        test=test_split.tolist(),
        docker_client=docker_client,
    )
    print(scores)
    collected_results.append(scores)
print("done")

In [ ]:
import pandas as pd

concated_results = pd.concat(collected_results)
pd.concat([concated_results, concated_results.apply(['mean'])])

## Resultados para sección evaluación

In [3]:
len(xss_dbs)

90

In [4]:
from sklearn.model_selection import KFold

# fixing random state to make this repeatable
kf = KFold(n_splits=5)
collected_results = []
base_results_dir = "/home/pablo/dev-results/xss-90-5fold-"
splits = list(kf.split(xss_dbs))

In [5]:
for i, (train_index, test_index) in enumerate(splits):
    print(f'split {i}: train: {len(train_index)} - test {len(test_index)}')

split 0: train: 72 - test 18
split 1: train: 72 - test 18
split 2: train: 72 - test 18
split 3: train: 72 - test 18
split 4: train: 72 - test 18


In [6]:
base_results_dir

'/home/pablo/dev-results/xss-90-5fold-'

In [8]:
import numpy as np

In [9]:
for i, (train_index, test_index) in enumerate(splits):
    print("Fold %d" % (i))
    res_dir = "%s%d" % (base_results_dir, i)
    train_split = np.take(xss_dbs, train_index)
    test_split = np.take(xss_dbs, test_index)

    scores = train_and_evaluate(
        config=config,
        results_dir=res_dir,
        query_type="xss",
        train=train_split.tolist(),
        test=test_split.tolist(),
        docker_client=docker_client,
    )
    print(scores)
print("done")

[INFO	2023-07-12 21:02:34,753] experiments.all	train_and_evaluate with train size 72, test size 18
[INFO	2023-07-12 21:02:34,754] experiments.all	Running training
[INFO	2023-07-12 21:02:34,754] scripts.docker	creating /home/pablo/dev-results/xss-90-5fold-0 dir. Omitting error if already existing
[INFO	2023-07-12 21:02:34,755] scripts.docker	creating /tmp/log dir. Omitting error if already existing


Fold 0


[INFO	2023-07-12 21:02:35,052] scripts.docker	running at container 11442ec0f8759e5d4b9f3b9931aaae099e01770da6855a1501864db81ed86c68. Use `docker logs 11442ec0f8759e5d4b9f3b9931aaae099e01770da6855a1501864db81ed86c68 --tail 10 --follow` to follow progress
[INFO	2023-07-12 22:19:33,978] scripts.docker	running combine scores
[INFO	2023-07-12 22:19:33,979] misc.combinescores	globbing folder: /home/pablo/dev-results/xss-90-5fold-0/*/reprScores.txt
[INFO	2023-07-12 22:19:33,981] misc.combinescores	working on 45 reprScores files
[INFO	2023-07-12 22:19:34,069] experiments.all	Running worse evaluation
[INFO	2023-07-12 22:19:34,070] scripts.evaluate	creating /home/pablo/dev-results/xss-90-5fold-0/worse dir. Omitting error if already existing
[INFO	2023-07-12 22:19:34,264] scripts.evaluate	Starting processing with 4 processes
[INFO	2023-07-12 22:19:34,281] scripts.evaluate	Evaluating ChangerHe - My-code
[INFO	2023-07-12 22:19:34,281] scripts.evaluate	Evaluating Breeze - breeze.js
[INFO	2023-07-12 

   precision  recall  accuracy  alerts to recover (atr)  alerts recovered  \
0        0.0     0.0       0.0                       19                 0   

   suprious alerts  projects with atr  avg atr per proj  
0                0                  4              4.75  
Fold 1


[INFO	2023-07-12 22:20:56,291] scripts.docker	running at container d38219d4b8ad630f7b54d6936c15763ee89ed6cb2f453a869a9b668ec8eec3dc. Use `docker logs d38219d4b8ad630f7b54d6936c15763ee89ed6cb2f453a869a9b668ec8eec3dc --tail 10 --follow` to follow progress
[INFO	2023-07-12 23:26:32,804] scripts.docker	running combine scores
[INFO	2023-07-12 23:26:32,806] misc.combinescores	globbing folder: /home/pablo/dev-results/xss-90-5fold-1/*/reprScores.txt
[INFO	2023-07-12 23:26:32,808] misc.combinescores	working on 46 reprScores files
[INFO	2023-07-12 23:26:32,922] experiments.all	Running worse evaluation
[INFO	2023-07-12 23:26:32,922] scripts.evaluate	creating /home/pablo/dev-results/xss-90-5fold-1/worse dir. Omitting error if already existing
[INFO	2023-07-12 23:26:33,110] scripts.evaluate	Starting processing with 4 processes
[INFO	2023-07-12 23:26:33,130] scripts.evaluate	Evaluating HelloZeroNet - ZeroNet
[INFO	2023-07-12 23:26:33,131] scripts.evaluate	Evaluating Huxpro - css-sucks-2015
[INFO	202

...

   precision    recall  accuracy  alerts to recover (atr)  alerts recovered  \
0        1.0  0.083333  0.083333                       12                 1   

   suprious alerts  projects with atr  avg atr per proj  
0                0                  4               3.0  
Fold 3


[INFO	2023-07-13 00:31:05,514] scripts.docker	running at container 13c977c63762089d94e779eb75bba923148dbf5800d0484c20563fb0076ca189. Use `docker logs 13c977c63762089d94e779eb75bba923148dbf5800d0484c20563fb0076ca189 --tail 10 --follow` to follow progress
[INFO	2023-07-13 01:34:32,712] scripts.docker	running combine scores
[INFO	2023-07-13 01:34:32,713] misc.combinescores	globbing folder: /home/pablo/dev-results/xss-90-5fold-3/*/reprScores.txt
[INFO	2023-07-13 01:34:32,715] misc.combinescores	working on 43 reprScores files
[INFO	2023-07-13 01:34:32,760] experiments.all	Running worse evaluation
[INFO	2023-07-13 01:34:32,760] scripts.evaluate	creating /home/pablo/dev-results/xss-90-5fold-3/worse dir. Omitting error if already existing
[INFO	2023-07-13 01:34:32,951] scripts.evaluate	Starting processing with 4 processes
[INFO	2023-07-13 01:34:32,974] scripts.evaluate	Evaluating Sean12697 - SlackChat
[INFO	2023-07-13 01:34:32,974] scripts.evaluate	Evaluating RetireJS - retire.js
[INFO	2023-07

   precision  recall  accuracy  alerts to recover (atr)  alerts recovered  \
0        0.0     0.0       0.0                        1                 0   

   suprious alerts  projects with atr  avg atr per proj  
0                0                  1               1.0  
Fold 4


[INFO	2023-07-13 01:39:29,194] scripts.docker	running at container 6b35b6abc94eefb3490f28b78a6ed3bcd4966578131a5331b5a7a6f78b008563. Use `docker logs 6b35b6abc94eefb3490f28b78a6ed3bcd4966578131a5331b5a7a6f78b008563 --tail 10 --follow` to follow progress
[INFO	2023-07-13 02:53:26,948] scripts.docker	running combine scores
[INFO	2023-07-13 02:53:26,949] misc.combinescores	globbing folder: /home/pablo/dev-results/xss-90-5fold-4/*/reprScores.txt
[INFO	2023-07-13 02:53:26,952] misc.combinescores	working on 46 reprScores files
[INFO	2023-07-13 02:53:27,006] experiments.all	Running worse evaluation
[INFO	2023-07-13 02:53:27,006] scripts.evaluate	creating /home/pablo/dev-results/xss-90-5fold-4/worse dir. Omitting error if already existing
[INFO	2023-07-13 02:53:27,194] scripts.evaluate	Starting processing with 4 processes
[INFO	2023-07-13 02:53:27,214] scripts.evaluate	Evaluating bpm - bpm
[INFO	2023-07-13 02:53:27,214] scripts.evaluate	Evaluating blockstrap - framework
[INFO	2023-07-13 02:53:

   precision  recall  accuracy  alerts to recover (atr)  alerts recovered  \
0        0.0     0.0       0.0                       12                 0   

   suprious alerts  projects with atr  avg atr per proj  
0                0                  4               3.0  
done


In [10]:
from scripts.calculate_scores import calculate_scores_df

collected_results = []
for i in range(5):
    df = calculate_scores_df("%s%d" % (base_results_dir, i), cleanup_base_dir="tmp")
    collected_results.append(df)

In [11]:
import pandas as pd

concated_results = pd.concat(collected_results)
concated_results = pd.concat([concated_results, concated_results.apply(['mean'])])
concated_results['name'] = [f'fold {i}' for i in range(5)] + ['mean']
concated_results

,precision,recall,accuracy,alerts to recover (atr),alerts recovered,suprious alerts,projects with atr,avg atr per proj,name
0,0.0,0.000000,0.000000,19.0,0.0,0.0,4.0,4.75,fold 0
0,0.0,0.000000,0.000000,12.0,0.0,0.0,5.0,2.40,fold 1
0,1.0,0.083333,0.083333,12.0,1.0,0.0,4.0,3.00,fold 2
0,0.0,0.000000,0.000000,1.0,0.0,0.0,1.0,1.00,fold 3
0,0.0,0.000000,0.000000,12.0,0.0,0.0,4.0,3.00,fold 4
mean,0.2,0.016667,0.016667,11.2,0.2,0.0,3.6,2.83,mean


In [12]:
with open("xss_90_results.csv", "w") as f:
    concated_results.to_csv(f, float_format='%.4f')

## prueba con kfold en 3

In [4]:
from sklearn.model_selection import KFold

# fixing random state to make this repeatable
kf = KFold(n_splits=3)
collected_results = []
base_results_dir = "/home/pablo/dev-results/xss-90-3fold-"
splits = list(kf.split(xss_dbs))

In [5]:
for i, (train_index, test_index) in enumerate(splits):
    print(f'split {i}: train: {len(train_index)} - test {len(test_index)}')

split 0: train: 60 - test 30
split 1: train: 60 - test 30
split 2: train: 60 - test 30


In [6]:
base_results_dir

'/home/pablo/dev-results/xss-90-3fold-'

In [7]:
import numpy as np

In [ ]:
for i, (train_index, test_index) in enumerate(splits):
    print("Fold %d" % (i))
    res_dir = "%s%d" % (base_results_dir, i)
    train_split = np.take(xss_dbs, train_index)
    test_split = np.take(xss_dbs, test_index)

    scores = train_and_evaluate(
        config=config,
        results_dir=res_dir,
        query_type="xss",
        train=train_split.tolist(),
        test=test_split.tolist(),
        docker_client=docker_client,
    )
    print(scores)
print("done")

[INFO	2023-07-28 11:21:20,574] experiments.all	train_and_evaluate with train size 60, test size 30
[INFO	2023-07-28 11:21:20,575] experiments.all	Running training
[INFO	2023-07-28 11:21:20,576] scripts.docker	creating /home/pablo/dev-results/xss-90-3fold-0 dir. Omitting error if already existing
[INFO	2023-07-28 11:21:20,576] scripts.docker	creating /tmp/log dir. Omitting error if already existing


Fold 0


[INFO	2023-07-28 11:21:21,243] scripts.docker	running at container 80095c776781a849e1b33063286879fa291abf62f6fb78df39aae4d3fec2dc24. Use `docker logs 80095c776781a849e1b33063286879fa291abf62f6fb78df39aae4d3fec2dc24 --tail 10 --follow` to follow progress


In [13]:
from scripts.calculate_scores import calculate_many_scores
calculate_many_scores(["/home/pablo/dev-results/xss-90-3fold-%d" % i for i, _ in enumerate(splits)])

,precision,recall,accuracy,atr,recovered,spurious,progwithatr,avgatr,predsinks,name
0,0.0,0.0,0.0,22,0,0,6,3.666667,160,fold 0
0,0.0,0.0,0.0,20,0,0,6,3.333333,182,fold 1
0,0.0,0.0,0.0,14,0,0,6,2.333333,158,fold 2
mean,0.0,0.0,0.0,18,0,0,6,3.111111,166,mean


Probando overfitting en XSS

In [ ]:
res_dir = "/tmp/xss-overfitting-3"
train_split = np.take(xss_dbs, splits[0][1]) # using test split since it might be smaller

scores = train_and_evaluate(
    config=config,
    results_dir=res_dir,
    query_type="xss",
    train=train_split.tolist(),
    test=train_split.tolist(),
    docker_client=docker_client,
)
scores

[INFO	2023-07-28 19:44:41,268] experiments.all	train_and_evaluate with train size 30, test size 30
[INFO	2023-07-28 19:44:41,269] experiments.all	Running training
[INFO	2023-07-28 19:44:41,269] scripts.docker	creating /tmp/xss-overfitting-3 dir. Omitting error if already existing
[INFO	2023-07-28 19:44:41,269] scripts.docker	creating /tmp/log dir. Omitting error if already existing
[INFO	2023-07-28 19:44:41,491] scripts.docker	running at container 70ef1fbbdf1911969c911fb205257e45505e0befcbd4b55753c18661b030e679. Use `docker logs 70ef1fbbdf1911969c911fb205257e45505e0befcbd4b55753c18661b030e679 --tail 10 --follow` to follow progress
